In [ ]:
from nanome.beta.redis_interface import PluginInstanceRedisInterface
from nanome.util.enums import StreamType
import os
import itertools

# Set up redis credentials
redis_host = os.environ.get("REDIS_HOST")
redis_port = os.environ.get("REDIS_PORT")
redis_password = os.environ.get("REDIS_PASSWORD")
redis_channel = ""  # Use room id code received when activating cookbook plugin

plugin = PluginInstanceRedisInterface(redis_host, redis_port, redis_password, redis_channel)
plugin.connect()
print('Plugin Connected')

In [ ]:
comps = plugin.request_complex_list()
[comp] = plugin.request_complexes([comps[0].index])
print(comp)

In [ ]:
roygbiv = [
    (255, 0, 0),  # Red
    (255, 127, 0),  # Orange
    (255, 255, 0),  # Yellow
    (0, 255, 0),  # Green
    (0, 0, 255),  # Blue
    (75, 0, 130),  # Indigo
    (148, 0, 211),  # Violet
]


def set_color_stream_data(atom_indices, color):
    """Set the color for every atom in the stream."""
    stream_data = [None] * (len(atom_indices) * 3)
    color_vals = itertools.cycle(color)
    for i in range(len(stream_data)):
        stream_data[i] = next(color_vals)
    return stream_data


# Create a writing stream to sets color for every atom in the complex.
atom_indices = [atom.index for atom in comp.atoms]
stream_type = StreamType.color
stream = plugin.create_writing_stream(atom_indices, stream_type)

color_cycle = itertools.cycle(roygbiv)
for color in color_cycle:
    stream_data = set_color_stream_data(atom_indices, color)
    plugin.update_stream(stream, stream_data)
